In [1]:
import torch
import numpy as np
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

import os
import sys
# sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('boostcamp'))))
# tmp = os.getcwd()
# os.chdir('../../../mmdetection/boostcamp/augmentation/augmixations')
# from augmixations.cutmix import SmartCutmix
# os.chdir(tmp)
# from augmentation.Cutmix_Augmentation import Cutmix_Augmentation

### model load

In [2]:
MODEL_PATH ='/opt/ml/detection/mmdetection/configs/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco.py'

MODEL = 'vfnet_resnest50_fpn_mdconv_c3-c5_mstrain_2x_coco'
#NAME = ''
WORK_DIR =MODEL
# config file 들고오기
cfg = Config.fromfile(MODEL_PATH)

# data config
dataset_type = 'CocoDataset'
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
num_classes = len(classes)

root = '/opt/ml/detection/dataset/'

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 4

cfg.data.train.classes= classes
cfg.data.train.ann_file = root+'kfold/cv_train1.json'
cfg.data.train.img_prefix = root


cfg.data.val.classes = classes
cfg.data.val.ann_file = root+'kfold/cv_val1.json'
cfg.data.val.img_prefix = root

cfg.data.test.classes = classes
cfg.data.test.ann_file = root+'test.json'
cfg.data.test.img_prefix = root

img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)


cfg.data.train.pipeline[2].img_scale = [(1024, 512), (1024, 768)]
cfg.data.train.pipeline[2].multiscale_mode = 'range'
cfg.data.val.pipeline[1].img_scale = [(512,512),(840,840),(1024, 1024)]
cfg.data.test.pipeline[1].img_scale = [(512,512),(840,840),(1024, 1024)]
# # for TTA
# cfg.data.test.pipeline[1].flip=True
# cfg.data.val.pipeline[1].flip=True

cfg.data.test.test_mode = True
#cfg.data.val.test_mode = True

cfg.data['train'] = dict(
        type='ClassBalancedDataset',
        oversample_thr=1e-3,
        dataset= cfg.data['train'])

# Build dataset
datasets = build_dataset(cfg.data.train)

# model config
cfg.model.bbox_head.num_classes = num_classes
cfg.model.test_cfg.score_thr =0
cfg.model.test_cfg.nms.type='soft_nms'



# cfg.model.neck =dict(
#         type='PAFPN',
#         in_channels=[256, 512, 1024, 2048],
#         out_channels=256,
#         start_level=1,
#         add_extra_convs ='on_output',
#         num_outs=5,
#         relu_before_extra_convs= True)

# weighted loss config , 분포 구하기
instance_count = np.zeros(num_classes).astype(int)
for idx in range(len(datasets)):
    label = datasets.dataset.get_ann_info(idx)['labels']
    unique, counts = np.unique(label, return_counts=True)
    
    # add the occurrence number to each class
    instance_count[unique] += counts
    
normedWeights = [1 - (x / sum(instance_count)) for x in instance_count]
normedWeights = torch.FloatTensor(normedWeights).cuda()

## loss에 등록
cfg.model.bbox_head.loss_cls.loss_weight = normedWeights


##ptimizer config
cfg.optimizer.lr =cfg.optimizer.lr * cfg.data.samples_per_gpu**(1/2)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

## other config
cfg.checkpoint_config = dict(interval=1)
cfg.seed = 1010
cfg.gpu_ids = [0]
cfg.work_dir = f'../work_dirs/{WORK_DIR}'

cfg.evaluation = dict(
    save_best='bbox_mAP',
    metric=['bbox']
)

# wandb
log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(
                project='p-stage_object_detection',
#                entity='boostcampaitech2-object-detection-level2-cv-03',
                name=f'{WORK_DIR}'))])
cfg.log_config = log_config

# learning policy
lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    gamma=0.2,
    step=[16, 22,28,34])
runner = dict(type='EpochBasedRunner', max_epochs=40)

# cfg.runner.max_epochs=40
# cfg.lr_config = dict(
#     policy='cyclic',
#     target_ratio=(10, 1e-4),
#     cyclic_times=1,
#     step_ratio_up=0.4,
# )
# cfg.momentum_config = dict(
#     policy='cyclic',
#     target_ratio=(0.85 / 0.95, 1),
#     cyclic_times=1,
#     step_ratio_up=0.4,
# )

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


### save config for test

In [3]:
import pickle
import os
os.makedirs(cfg.work_dir,exist_ok=True)
with open(os.path.join(cfg.work_dir,'cfg.pkl'),'wb') as f:
    pickle.dump(cfg,f)

## train

In [4]:
# Build the detector
model = build_detector(cfg.model)
model.init_weights()

/opt/conda/envs/detection/lib/python3.7/site-packages/mmdet/core/anchor/builder.py:17: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
2021-10-01 10:21:35,127 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2021-10-01 10:21:35,128 - mmcv - INFO - load model from: torchvision://resnet50
2021-10-01 10:21:35,131 - mmcv - INFO - Use load_from_torchvision loader
2021-10-01 10:21:35,414 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.conv_offset.weight, layer2.0.conv2.conv_offset.bias, layer2.1.conv2.conv_offset.weight, layer2.1.conv2.conv_offset.bias, layer2.2.conv2.conv_offset.weight, layer2.2.conv2.conv_offset.bias, layer2.3.conv2.conv_offset.weight, layer2.3.conv2.conv_offs

2021-10-01 10:21:35,608 - mmcv - INFO - 
backbone.layer2.0.bn3.weight - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,608 - mmcv - INFO - 
backbone.layer2.0.bn3.bias - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,609 - mmcv - INFO - 
backbone.layer2.0.downsample.0.weight - torch.Size([512, 256, 1, 1]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,610 - mmcv - INFO - 
backbone.layer2.0.downsample.1.weight - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,611 - mmcv - INFO - 
backbone.layer2.0.downsample.1.bias - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,612 - mmcv - INFO - 
backbone.layer2.1.conv1.weight - torch.Size([128, 512, 1, 1]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,613 - mmcv - INFO - 
backbone.layer2.1.bn1.weight - torch.Size([128])

2021-10-01 10:21:35,655 - mmcv - INFO - 
backbone.layer3.1.bn1.bias - torch.Size([256]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,655 - mmcv - INFO - 
backbone.layer3.1.conv2.weight - torch.Size([256, 256, 3, 3]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,656 - mmcv - INFO - 
backbone.layer3.1.conv2.conv_offset.weight - torch.Size([27, 256, 3, 3]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-10-01 10:21:35,656 - mmcv - INFO - 
backbone.layer3.1.conv2.conv_offset.bias - torch.Size([27]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-10-01 10:21:35,657 - mmcv - INFO - 
backbone.layer3.1.bn2.weight - torch.Size([256]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,657 - mmcv - INFO - 
backbone.layer3.1.bn2.bias - torch.Size([256]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,658 - mmcv - INFO - 
backb

2021-10-01 10:21:35,686 - mmcv - INFO - 
backbone.layer4.0.conv1.weight - torch.Size([512, 1024, 1, 1]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,687 - mmcv - INFO - 
backbone.layer4.0.bn1.weight - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,688 - mmcv - INFO - 
backbone.layer4.0.bn1.bias - torch.Size([512]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,688 - mmcv - INFO - 
backbone.layer4.0.conv2.weight - torch.Size([512, 512, 3, 3]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-10-01 10:21:35,689 - mmcv - INFO - 
backbone.layer4.0.conv2.conv_offset.weight - torch.Size([27, 512, 3, 3]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-10-01 10:21:35,690 - mmcv - INFO - 
backbone.layer4.0.conv2.conv_offset.bias - torch.Size([27]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-10-01 10:21:35,690 - mmcv

2021-10-01 10:21:35,721 - mmcv - INFO - 
bbox_head.cls_convs.0.gn.weight - torch.Size([256]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-10-01 10:21:35,721 - mmcv - INFO - 
bbox_head.cls_convs.0.gn.bias - torch.Size([256]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-10-01 10:21:35,722 - mmcv - INFO - 
bbox_head.cls_convs.1.conv.weight - torch.Size([256, 256, 3, 3]): 
NormalInit: mean=0, std=0.01, bias=0 
 
2021-10-01 10:21:35,722 - mmcv - INFO - 
bbox_head.cls_convs.1.gn.weight - torch.Size([256]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-10-01 10:21:35,723 - mmcv - INFO - 
bbox_head.cls_convs.1.gn.bias - torch.Size([256]): 
The value is the same before and after calling `init_weights` of VFNet  
 
2021-10-01 10:21:35,723 - mmcv - INFO - 
bbox_head.cls_convs.2.conv.weight - torch.Size([256, 256, 3, 3]): 
The value is the same before and after calling `init_weights` of VFN

In [5]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

2021-10-01 10:21:36,960 - mmdet - INFO - Start running, host: root@2f4df6ced18c, work_dir: /opt/ml/detection/mmdetection/boostcamp/work_dirs/vfnet_resnest50_fpn_mdconv_c3-c5_mstrain_2x_coco
2021-10-01 10:21:36,962 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/runner/hooks/evaluation.py:192: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')
wandb: Currently logged in as: larcane (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/opt/conda/envs/detection/lib/python3.7/site-packages/mmdet/core/anchor/anchor_generator.py:324: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/opt/conda/envs/detection/lib/python3.7/site-packages/mmdet/core/anchor/anchor_generator.py:361: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '
2021-10-01 10:21:56,747 - mmdet - INFO - Epoch [1][10/490]	lr: 3.287e-03, eta: 4:08:42, time: 1.270, data_time: 0.292, memory: 7035, loss_cls: 0.5648, loss_bbox: 0.9440, loss_bbox_rf: 1.3265, loss: 2.8353, grad_norm: 13.0649
2021-10-01 10:22:06,298 - mmdet - INFO - Epoch [1][20/490]	lr: 3.796e-03, eta: 3:37:41, time: 0.955, data_time: 0.019, memory: 7035, loss_cls: 0.7623, loss_bbox: 0.7937, loss_bbox_rf: 1.0809, loss: 2.6368, grad_norm: 6.4864
2021-10-01 10:22:15,862 - mmdet - INFO -

KeyboardInterrupt: 